Imports and data preparation

In [12]:
%matplotlib inline
import pandas as pd
import quandl , math ,datetime
import numpy as np
from sklearn import preprocessing , model_selection , svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

import pickle


df = pd.read_csv("/home/balerion/Desktop/Ai/GOOGL.csv")

# print(df.head())

df = df[["Open" , "High" , "Low" , "Close" , "Volume"]]

# df["HL_PCT"] = (df["High"] - df["Low"]) / df["Low"] * 100

df["HC_PCT"] = (df["High"] - df["Close"]) / df["Close"] * 100.0

df["PCT_change"] = (df["Close"] - df["Open"]) / df["Open"] * 100.0

df = df[["Close" , "HC_PCT" , "PCT_change" , "Volume"]]

# print(df.head())


forecast_col = "Close"

df.fillna(-99999 , inplace=True) # to get rid of nill data

forecast_out = int(math.ceil(0.01*len(df))) # for every 40 odd days

print(forecast_out)

df["label"] = df[forecast_col].shift(-forecast_out)

# print(df.head())



45


Prprocessing and Scaling

In [13]:
x = np.array(df.drop(columns=["label"]))# x = np.array(df.drop(columns = ["label"])) -> contains features
x = preprocessing.scale(x)
x = x[:-forecast_out]

x_lately = x[-forecast_out:]

# standardization of data x' = x - mean / standard deviation
# Standardized data refers to data that has been transformed to have a mean of 0 and a standard deviation of 1
# x = x[:-forecast_out+1]

df.dropna(inplace=True)
y = np.array(df["label"])
# y = np.array(df["label"])

print(len(x) , len(y))



4386 4386


Why Scaling is important ?


In [14]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import StandardScaler

# # Sample DataFrame
# data = {
#     'feature1': [1, 2, 3, 4, 5],
#     'feature2': [10, 20, 30, 40, 50],
#     'feature3': [100, 200, 300, 400, 500]
# }
# df = pd.DataFrame(data)

# # Extract features
# x = df.values

# # Standardize the features
# scaler = StandardScaler()
# x_standardized = scaler.fit_transform(x)

# print("Original data:\n", x)
# print("Standardized data:\n", x_standardized)


#  Q. Why Standardization is Important ?

#  -> Improved Algorithm Performance:
# Many machine learning algorithms, such as those using gradient descent (e.g., linear regression, logistic regression) or distance metrics (e.g., k-nearest neighbors, support vector machines), perform better or converge faster with standardized data.

#  -> Fair Feature Contribution: 
# Standardization ensures that each feature contributes equally to the model. Without standardization, features with larger ranges can dominate the learning process, leading to biased results.

#  -> Stable and Faster Convergence: 
# For optimization algorithms, such as gradient descent, standardized data ensures stable and faster convergence by providing a smoother cost function landscape.

Model training and testing

In [15]:
x_train , x_test , y_train , y_test = model_selection.train_test_split(x , y , test_size=0.2)
# classifier =  LinearRegression(n_jobs=10)   #svm.SVR(kernel = "poly")

# classifier.fit(x_train , y_train)  # fit("used for training the model")

# with open('linearregression.pickle' , 'wb') as f:
#     pickle.dump(classifier , f)

pickle_in = open('linearregression.pickle' , 'rb')
classifier = pickle.load(pickle_in)

accuracy = classifier.score(x_test , y_test) # score("used for testing accuracy")

forecast_set = classifier.predict(x_lately)

print(forecast_set , accuracy , forecast_out)

# print(accuracy)


[3070.20186253 3057.76373462 3059.89394874 3102.69001132 3083.46844312
 3032.5719515  3013.87957305 3019.85169202 2943.24376126 3014.05083121
 2944.98027895 2933.61082466 2962.86701826 2940.81994157 2960.02269269
 3047.19910305 3062.53946802 3057.28624551 3059.32741329 3017.64821988
 2973.84848507 3034.64129759 2993.35272201 2935.88618333 2929.89360021
 2969.09935988 3034.35812729 3042.65786012 3058.71409394 3033.70423519
 3032.77079466 3023.95609818 2998.48177828 2998.40877962 2990.92090633
 2865.83503898 2860.3827443  2834.10569857 2875.63751123 2893.74622333
 2928.3035247  2878.99440589 2895.93482151 2814.19368991 2803.56209611] 0.9813245097827078 45


What is SVR ?

In [5]:
# Key Concepts of SVR

# Margin of Tolerance : SVR introduces a margin of tolerance, within which errors are ignored. The goal is to find a function that deviates from the actual data points by a value no greater than tolerance.

# Support Vectors: Data points that lie within the margin boundaries and influence the position and orientation of the regression line.

# Kernel Trick: SVR can use kernel functions (like linear, polynomial, or RBF) to handle non-linear relationships by mapping input features into higher-dimensional spaces.